In [5]:
import sys
print(sys.getdefaultencoding())

utf-8


In [1]:
# DELG feature extractor
# https://github.com/tensorflow/models/blob/master/research/delf/delf/python/delg/DELG_INSTRUCTIONS.md
import os

os.makedirs('delg/data', exist_ok=True)
os.chdir('delg/data')

In [2]:
import urllib.request
import tarfile

# Oxford 데이터셋 다운로드
oxford_url = "http://www.robots.ox.ac.uk/~vgg/data/oxbuildings/oxbuild_images.tgz"
urllib.request.urlretrieve(oxford_url, "oxbuild_images.tgz")

# 압축 해제
with tarfile.open("oxbuild_images.tgz") as tar:
    tar.extractall("oxford5k_images/")

In [3]:
# Paris 데이터셋 다운로드
paris_urls = [
    "http://www.robots.ox.ac.uk/~vgg/data/parisbuildings/paris_1.tgz",
    "http://www.robots.ox.ac.uk/~vgg/data/parisbuildings/paris_2.tgz"
]

for url in paris_urls:
    file_name = url.split("/")[-1]
    urllib.request.urlretrieve(url, file_name)

# 압축 해제
os.makedirs("paris6k_images_tmp", exist_ok=True)

with tarfile.open("paris_1.tgz") as tar:
    tar.extractall("paris6k_images_tmp/")
with tarfile.open("paris_2.tgz") as tar:
    tar.extractall("paris6k_images_tmp/")


In [4]:
# 모든 이미지 같은 디렉토리로 이동
os.makedirs("paris6k_images", exist_ok=True)
for root, dirs, files in os.walk("paris6k_images_tmp/paris"):
    for file in files:
        os.rename(os.path.join(root, file), os.path.join("paris6k_images", file))

In [5]:
#Revisited Annotations
revisited_urls = [
    "http://cmp.felk.cvut.cz/revisitop/data/datasets/roxford5k/gnd_roxford5k.mat",
    "http://cmp.felk.cvut.cz/revisitop/data/datasets/rparis6k/gnd_rparis6k.mat",
    "http://cmp.felk.cvut.cz/cnnimageretrieval/data/test/roxford5k/gnd_roxford5k.pkl",
    "http://cmp.felk.cvut.cz/cnnimageretrieval/data/test/rparis6k/gnd_rparis6k.pkl"
]

for url in revisited_urls:
    file_name = url.split("/")[-1]
    urllib.request.urlretrieve(url, file_name)

In [6]:
# 모델 다운로드
model_url = "http://storage.googleapis.com/delf/r50delg_gld_20200814.tar.gz"
urllib.request.urlretrieve(model_url, "r50delg_gld_20200814.tar.gz")

# 압축 해제
os.makedirs("parameters", exist_ok=True)
with tarfile.open("r50delg_gld_20200814.tar.gz") as tar:
    tar.extractall("parameters/")

pwd

In [113]:
import sys
sys.path.insert(0, 'D:/coding/Python_Project/DELG/models/research/delf/delf')

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import io  # io 모듈: utf-8로

from absl import app
from absl import flags
import numpy as np
import tensorflow as tf

from google.protobuf import text_format
from protos import delf_config_pb2
from python import datum_io
from python import feature_io
from python import utils
from python.datasets.revisited_op import dataset
from python.examples import extractor
from PIL import Image  # 추가: Image 모듈을 사용해 PIL 이미지로 로드하기

# feature 저장할 경로 설정하기 (FLAGS 대신 사용함)
delf_config_path = 'D:/coding/Python_Project/DELG/models/research/delf/delf/python/delg/r50delg_gld_config.pbtxt'  # FLAGS.delf_config_path
# dataset_file_path = 'D:/coding/Python_Project/DELG/gnd_roxford5k.mat'  # FLAGS.dataset_file_path # 단순히 이미지 간 유사도를 비교하는 작업이므로 .mat없이 특징을 추출하면 됨
images_dir = 'D:/coding/Python_Project/Crop/yolov5/sample_dataset/cropped_images'  # FLAGS.images_dir
output_features_dir = 'D:/coding/Python_Project/DELG/features'  # FLAGS.output_features_dir

# 확장자 설정 (FLAGS 대신 상수로 정의함)
_DELG_GLOBAL_EXTENSION = '.delg_global'
_DELG_LOCAL_EXTENSION = '.delg_local'
_IMAGE_EXTENSION = '.jpg'

# log status 알림 주기 (FLAGS 대신 상수로 정의함)
_STATUS_CHECK_ITERATIONS = 50 # 이미지 50개마다 알림

def main(argv):
    if len(argv) > 1:
        raise RuntimeError('Too many command-line arguments.')

    # 데이터셋 파일로부터 이미지 목록 읽기 - .mat 사용 안함
#     print('Reading list of images from dataset file...')
#     query_list, index_list, ground_truth = dataset.ReadDatasetFile(dataset_file_path)
    
#     if image_set == 'query':
#         image_list = query_list
#     else:
#         image_list = index_list

#     num_images = len(image_list)
#     print(f'done! Found {num_images} images')

    _IMAGE_EXTENSION = ['.jpg', '.png'] # png도 확장자에 포함

    # 이미지 경로 list 만들기
    image_list = []
    for root, dirs, files in os.walk(images_dir):
        for file in files:
            if any(file.endswith(ext) for ext in _IMAGE_EXTENSION):
                relative_path = os.path.relpath(os.path.join(root, file), images_dir)
                image_list.append(relative_path)
                
    num_images = len(image_list)
    print(f'done! Found {num_images} images')

    # DelfConfig proto 파싱
    config = delf_config_pb2.DelfConfig()
    with io.open(delf_config_path, 'r', encoding='utf-8') as f:
        text_format.Parse(f.read(), config)

    # output 디렉토리 생성
    if not tf.io.gfile.exists(output_features_dir):
        tf.io.gfile.makedirs(output_features_dir)

    extractor_fn = extractor.MakeExtractor(config)

    start = time.time()

    for i, image_name in enumerate(image_list):
        if i == 0:
            print('Starting to extract features...')
        elif i % _STATUS_CHECK_ITERATIONS == 0:
            elapsed = time.time() - start
            print(f'Processing image {i+1} out of {num_images}, last {_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds')
            start = time.time()

        input_image_filename = os.path.join(images_dir, image_name)

        # output 디렉토리를 생성
        output_global_feature_dir = os.path.join(output_features_dir, os.path.dirname(image_name))
        output_local_feature_dir = os.path.join(output_features_dir, os.path.dirname(image_name))

        should_skip_global = True
        should_skip_local = True

        if config.use_global_features:
            output_global_feature_filename = os.path.join(output_global_feature_dir, os.path.basename(image_name) + _DELG_GLOBAL_EXTENSION)
            if not tf.io.gfile.exists(output_global_feature_filename):
                should_skip_global = False

        if config.use_local_features:
            output_local_feature_filename = os.path.join(output_local_feature_dir, os.path.basename(image_name) + _DELG_LOCAL_EXTENSION)
            if not tf.io.gfile.exists(output_local_feature_filename):
                should_skip_local = False

        if should_skip_global and should_skip_local:
            print(f'Skipping {image_name}')
            continue

        # 이미지 로딩 및 특징 추출
        with open(input_image_filename, 'rb') as f:
            pil_im = Image.open(io.BytesIO(f.read()))
            pil_im = pil_im.convert('RGB')  # 이미지를 RGB로 변환

        resize_factor = 1.0
        im = np.array(pil_im)

        # 특징 추출 및 저장하기
        extracted_features = extractor_fn(im, resize_factor)
        
        if config.use_global_features:
            if not tf.io.gfile.exists(output_global_feature_dir):
                tf.io.gfile.makedirs(output_global_feature_dir)
            global_descriptor = extracted_features['global_descriptor']
            datum_io.WriteToFile(global_descriptor, output_global_feature_filename)
        
        if config.use_local_features:
            if not tf.io.gfile.exists(output_local_feature_dir):
                tf.io.gfile.makedirs(output_local_feature_dir)
            locations = extracted_features['local_features']['locations'] # locations: feature 위치
            descriptors = extracted_features['local_features']['descriptors'] # descriptors: feature descriptor
            feature_scales = extracted_features['local_features']['scales'] # scales: feature 크기
            attention = extracted_features['local_features']['attention'] # attention: attention 점수. 얼마나 집중되었는가 점수로.
            feature_io.WriteToFile(output_local_feature_filename, locations, feature_scales, descriptors, attention) # 넘파이 ndarray로 값이 전부 0으로 넘어가는데 뭔진 모르겠음

if __name__ == '__main__':
    main([])

done! Found 820 images
Starting to extract features...
Processing image 51 out of 820, last 50 images took 97.69883179664612 seconds
Processing image 101 out of 820, last 50 images took 137.55818700790405 seconds
Processing image 151 out of 820, last 50 images took 220.6449592113495 seconds
Processing image 201 out of 820, last 50 images took 109.34353613853455 seconds
Processing image 251 out of 820, last 50 images took 116.55162334442139 seconds
Processing image 301 out of 820, last 50 images took 84.52076411247253 seconds
Processing image 351 out of 820, last 50 images took 200.92803502082825 seconds
Processing image 401 out of 820, last 50 images took 209.58974194526672 seconds
Processing image 451 out of 820, last 50 images took 544.560751914978 seconds
Processing image 501 out of 820, last 50 images took 105.51707911491394 seconds
Processing image 551 out of 820, last 50 images took 314.20444893836975 seconds
Processing image 601 out of 820, last 50 images took 679.9912004470825 

In [106]:
# 에러발생: cannot import name 'runtime_version' from 'google.protobuf' (C:\Users\User\anaconda3\Lib\site-packages\google\protobuf\__init__.py)
# protobuf 버전 호환 문제 => 최신 버전으로 업데이트함
# pip install --upgrade protobuf

# Requirement already satisfied: protobuf in c:\users\user\anaconda3\lib\site-packages (3.20.3)
# Collecting protobuf
#   Using cached protobuf-5.27.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
# Using cached protobuf-5.27.3-cp310-abi3-win_amd64.whl (426 kB)
# Installing collected packages: protobuf
#   Attempting uninstall: protobuf
#     Found existing installation: protobuf 3.20.3
#     Uninstalling protobuf-3.20.3:
#       Successfully uninstalled protobuf-3.20.3
# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
# tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.27.3 which is incompatible.
# Successfully installed protobuf-5.27.3
# 버전 문제는 없어졌다. 근데 설치과정에서 위와 같은 에러가 났다.
# 위 에러는 pip install protobuf==3.20.3 를 하면 해결된다.

In [114]:
# 이번엔 ModuleNotFoundError: No module named 'delf.python'
# path 설정을 다시 해준다.

In [115]:
# 시스템 path 확인
import sys
print(sys.path)

['D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/delf', 'D:/coding/Python_Project/DELG/models/research/delf/del